In [9]:
#/home/pui_user/PUI2018_msm796/HW4_msm796
#declare libraries
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
import datetime
from dateutil.relativedelta import relativedelta #DOWNLOAD THIS PACKAGE#
import os
import requests #DOWNLOAD THIS PACKAGE#
import zipfile #DOWNLOAD THIS PACKAGE#
import io #DOWNLOAD THIS PACKAGE#
import pandas as pd

#library#
    
#declare parameters
puidata = os.getenv("PUIDATA")
if puidata is None:
    os.environ["PUIDATA"] = "%s/PUIdata"%os.getenv("HOME")
    puidata = os.getenv("PUIDATA")

#declare helper functions
def download_citibike_data(yearstart,yearend,monthstart,monthend,output_path):
    '''
    change log:
    
    version 1.0 - Downloads citibike datasets zip files and unzips them. does checking if file already
    exists in path. currently has no input error handling logic.
    ---------------------------------------------------------------------------------------------
    Parameters:
    name - type - description
    yearstart - <int> - refers to the year where you want your download to start
    yearend - <int> - refers to the year where you want your download to start
    monthstart - <int> - refers to the month ssociated with yearstart where you want your download to start
    monthend - <int> - refers to the month associated with yearend where you want your download to start
    outputpath - <str> - refers to location where you want to dump the csv files
    
    example usage:
    download_citibike_data(2013,2018,6,8,puidata)
    - this downloads the citibike datasets from June 2013 to August 2018. Files will be unzipped at
    the puidata location.
    
    required libraries:
    datautil - required for year & month generation
    requests - for downloading the zip file
    zipfile - to unzip the file
    io - to read binary data and give it to zipfile; effectly keeping the entire process in memory
    '''
    #might include in error handling in the future for incorrect input
    curr_date = start_date=datetime.datetime(yearstart,monthstart,1).date()
    end_date=datetime.datetime(yearend,monthend,1).date()
    year_month=[start_date.strftime('%Y%m')]
    base_citi_url='https://s3.amazonaws.com/tripdata/'
    while curr_date < end_date:
        curr_date += relativedelta(months=1)
        year_month.append(curr_date.strftime('%Y%m'))
    for x in year_month:
    #need to change logic as csv filename convention isn't done properly#
    #will just base on dates found in directory#
        filename=x+"-citibike-tripdata.csv"
        if os.path.isfile(output_path+"/"+filename)==True:
            print("{} already exists!".format(filename))
        else:
            print("Downloading {}".format(filename))
            year=int(x[:4])
            if year>=2017:
                ending=".csv.zip"
            else:
                ending=".zip"
            url=base_citi_url+x+"-citibike-tripdata"+ending
            r = requests.get(url, stream=True)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall(output_path)
            
def append_all_citi_bike_data(input_path):
    '''
    change log:
    
    version 1.0 - <stuff>. Does checking if file already
    exists in path. currently has no input error handling logic.

    Parameters:
    name - type - description
    
    input_path - <str> - location where the csv files are stored
    -Naively appends all files into one superfile
    
    required libraries:
    pandas - the output will be a dataframe, also used to load the files
    '''
    #currently implementing
    files_directory=os.listdir(input_path)
    csv_files = [x for x in files_directory if ".csv" in x]
    super_csv = pd.concat([pd.read_csv(input_path+"/"+f) for f in csv_files])
    return super_csv

In [6]:
files_directory=os.listdir(puidata)
csv_files = [x for x in files_directory if ".csv" in x]
print(csv_files)

['201705-citibike-tripdata.csv', '201507-citibike-tripdata.csv', '201601-citibike-tripdata.csv', '201411-citibike-tripdata.csv', '201702-citibike-tripdata.csv', '201512-citibike-tripdata.csv', '2013-12 - Citi Bike trip data.csv', '201704-citibike-tripdata.csv', '201606-citibike-tripdata.csv', '201710-citibike-tripdata.csv', '201607-citibike-tripdata.csv', '2014-05 - Citi Bike trip data.csv', '201504-citibike-tripdata.csv', '201509-citibike-tripdata.csv', '2013-07 - Citi Bike trip data.csv', '2013-08 - Citi Bike trip data.csv', '201806-citibike-tripdata.csv', '201711-citibike-tripdata.csv', '201410-citibike-tripdata.csv', '2014-04 - Citi Bike trip data.csv', '2013-09 - Citi Bike trip data.csv', '201802-citibike-tripdata.csv', '2014-03 - Citi Bike trip data.csv', '201808-citibike-tripdata.csv', '2013-11 - Citi Bike trip data.csv', '201306-citibike-tripdata.csv', '201503-citibike-tripdata.csv', '201610-citibike-tripdata.csv', '2014-08 - Citi Bike trip data.csv', '2014-06 - Citi Bike trip 

In [ ]:
#downloads from beginning to latest available as of late September
#download_citibike_data(2013,2018,6,8,puidata)
complete_set=append_all_citi_bike_data(puidata)